In [24]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
from sklearn.metrics import (
    mean_squared_error, r2_score, accuracy_score, f1_score, confusion_matrix
)
from scipy.stats import pearsonr

##  Max-Max gcPBM Data Processing Workflow

### 🧪 STEP 1 — Load Input Data

- `gcPBM_probe_sequence.txt` — contains just the 36 bp sequences.
Obtained from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GPL17173

- `gcPBM_max_normalized_intensity.txt` — contains the raw Alexa488 intensity values.
Obtained from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1447392

- `GSM2746660_Max_8mers_11111111.txt` - contains the Universal PBM 8-mer E-scores
Obtained from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2746660

---

### 🧹 STEP 2 — Filter Probes

Remove non-specific probes:

- Use `GSM2746660_Max_8mers_1111111.txt` (E-score file).
- Discard any probe whose **flanking regions** (outside central 36 bp) contain **8-mers with E-score ≥ 0.3**.

> This ensures probe specificity by excluding probes with high-affinity off-target 8-mers.

---

### 📊 STEP 3 — Stratify Probes

Based on **normalized intensities**, classify probes into binding strength categories:

- `78` **Unbound** — lowest intensity values.
- `45` **Weak** — middle-range values.
- `45` **Strong** — highest intensity values.

> Why are these categories not equal?

---

### 💾 STEP 4 — Save the Final Dataset

Create three separate files with the **36 bp sequences** corresponding to each category:

- `unbound.txt`
- `weak.txt`
- `strong.txt`

These files serve as inputs for downstream machine learning models and binding simulations.


## 1. **Load Input Data**  

In [25]:
# Universal PBM 8-mer E-scores (two columns of 8-mers)
pbm_8mer = pd.read_csv(
    'GSM2746660_Max_8mers_11111111.txt',
    sep='\s+', comment='#', header=0)

# gcPBM probe sequences and normalized intensities
gcPBM_norm_intensity = pd.read_csv(
    'gcPBM_max_normalized_intensity.txt',
    sep='\s+', comment='#', header=0)
gcPBM_probe_seq = pd.read_csv(
    'gcPBM_probe_sequence.txt',
    sep='\s+', comment='#', header=0)

## 2. **Filter Probes**  

In [26]:
# Merge on ID and clean up
gcPBM = (
    pd.merge(gcPBM_probe_seq, gcPBM_norm_intensity,
             left_on='ID', right_on='ID_REF')
      .drop('ID_REF', axis=1)
      .set_index('ID')
      .rename(columns={'VALUE':'Mean Intensity'})
)

# Filter to only Myc-bound and Myc-unbound probes
gcPBM_myc = gcPBM[
    gcPBM['Name']
         .apply(lambda x: ('Bound' in x) or ('Unbound_Myc' in x))
]

In [27]:
gcPBM

,Name,SEQUENCE,Mean Intensity
ID,,,
1,Bound_000001_100_100,CCCACCCGCCGCGCCCACGCGTGCCGCGGTCGCTTG,10934
2,Bound_000002_100_100,CGGACCCTGGCTCTCCACGCGCCTCCGACATGGCGG,7835
3,Bound_000003_000_100,GAAGAGGACGGCATGCATGCGGACCCCGTCCACCAC,11037
4,Bound_000004_001_001,CCGGCCGGCTGTGGCCACGCGTCTGCGCATGCGCGC,10758
5,Bound_000005_010_000,ATGTACACATGTACCCACATGCACACAGAAACACAC,18501
...,...,...,...
29333,Test12mer_013,GTCAAGAGCAGTATGCACGTGTTGAGCTGGCTAAGG,20554
29334,Test12mer_014,GTCAAGAGCAGTACGCACGTGGTGAGCTGGCTAAGG,26733
29335,Test12mer_015,GTCAAGAGCAGTCGGCACGTGGCAAGCTGGCTAAGG,18705


In [28]:
pbm_8mer_dict = pd.Series(
    pbm_8mer['E-score'].values,
    index=pbm_8mer['8-mer']
).to_dict()
# also include second column of 8-mers
pbm_8mer_dict.update(pd.Series(
    pbm_8mer['E-score'].values,
    index=pbm_8mer['8-mer.1']
).to_dict())

print("Total Myc probes:", gcPBM_myc.shape)

Total Myc probes: (28463, 3)


In [29]:
def check_flank(left, right, escore_dict):
    """Return False if any 8-mer in flanks has E-score ≥ 0.3."""
    # scan left flank
    for i in range(len(left) - 7):
        if escore_dict.get(left[i:i+8], 0) >= 0.3:
            return False
    # scan right flank
    for i in range(len(right) - 7):
        if escore_dict.get(right[i:i+8], 0) >= 0.3:
            return False
    return True

gcPBM_myc_flank = gcPBM_myc[
    gcPBM_myc['SEQUENCE']
       .apply(lambda s: check_flank(s[:14], s[22:], pbm_8mer_dict))
]
print("After flank filter:", gcPBM_myc_flank.shape)

After flank filter: (19954, 3)


In [30]:
def check_center(seq, escore_dict):
    """Ensure central 8-mer is top among adjacent, and no outside 8-mer beats them."""
    c8  = seq[14:22]
    al  = seq[13:21]
    ar  = seq[15:23]
    ec, el, er = escore_dict.get(c8,0), escore_dict.get(al,0), escore_dict.get(ar,0)
    # must beat both adjacent
    if not (ec>el and ec>er):
        return False
    # no outside 8-mer should exceed adjacent scores
    for i in range(len(seq)-7):
        if i<13 or i>15:
            e = escore_dict.get(seq[i:i+8],0)
            if e>el or e>er:
                return False
    return True

gcPBM_myc_flank_center = gcPBM_myc_flank[
    gcPBM_myc_flank['SEQUENCE']
       .apply(lambda s: check_center(s, pbm_8mer_dict))
]
print("After center filter:", gcPBM_myc_flank_center.shape)

After center filter: (8973, 3)


In [31]:
# Keep unbound probes for later concatenation
gcPBM_myc_flank_unbound = gcPBM_myc_flank[
    gcPBM_myc_flank['Name'].str.contains('Unbound')
]

# Combine bound (center-filtered) and unbound
gcPBM_myc_final = pd.concat([
    gcPBM_myc_flank_unbound,
    gcPBM_myc_flank_center
])

In [32]:
# Annotate categories and compute log intensity
gcPBM_myc_final['Probe'] = gcPBM_myc_final['Name'].apply(
    lambda x: 'Unbound' if 'Unbound' in x else 'Bound'
)
gcPBM_myc_final['Log Intensity'] = np.log(
    gcPBM_myc_final['Mean Intensity']
)

# Extract k-mer substrings for modeling
gcPBM_myc_final['12mer'] = gcPBM_myc_final['SEQUENCE'].str[12:24]
gcPBM_myc_final['10mer'] = gcPBM_myc_final['SEQUENCE'].str[13:23]
gcPBM_myc_final['8mer']  = gcPBM_myc_final['SEQUENCE'].str[14:22]
gcPBM_myc_final['6mer']  = gcPBM_myc_final['SEQUENCE'].str[15:21]

gcPBM_myc_final

,Name,SEQUENCE,Mean Intensity,Probe,Log Intensity,12mer,10mer,8mer,6mer
ID,,,,,,,,,
28164,Unbound_Myc1_001,TCCAGCAAACTTTTCTTTGTTCGCTGCAGTGCCGCC,3665,Unbound,8.206584,TTCTTTGTTCGC,TCTTTGTTCG,CTTTGTTC,TTTGTT
28165,Unbound_Myc1_002,GGGCTCCCAGGGGCTGCTGCTGCCTTTCCGGTCTTG,2760,Unbound,7.922986,GCTGCTGCTGCC,CTGCTGCTGC,TGCTGCTG,GCTGCT
28166,Unbound_Myc1_003,TATGGAGGATAACTTGGTACAGCCCCTATAGAGGGC,3854,Unbound,8.256867,CTTGGTACAGCC,TTGGTACAGC,TGGTACAG,GGTACA
28167,Unbound_Myc1_004,GGAATCTGAGGGATGAAGACTCAGGTCAGCAGGCTG,3411,Unbound,8.134761,ATGAAGACTCAG,TGAAGACTCA,GAAGACTC,AAGACT
28168,Unbound_Myc1_005,AGCGCACTTCAGGGAACCGCACTTAGGGTCCGATGG,5099,Unbound,8.536800,GGAACCGCACTT,GAACCGCACT,AACCGCAC,ACCGCA
...,...,...,...,...,...,...,...,...,...
28148,Bound_028148_001_000,GCCTGCGGCGGCCCCCACGCGCGTACTCACGGAGCT,6470,Bound,8.774931,CCCCACGCGCGT,CCCACGCGCG,CCACGCGC,CACGCG
28150,Bound_028150_111_000,TGGTTATAAACACTCCATGTGGTTTTGTTTGGTGGT,15251,Bound,9.632400,CTCCATGTGGTT,TCCATGTGGT,CCATGTGG,CATGTG
28154,Bound_028154_001_000,GACCCGCGCTGCGCCCACGTGCAGGGCCCGACCCCC,22287,Bound,10.011759,GCCCACGTGCAG,CCCACGTGCA,CCACGTGC,CACGTG


In [33]:
# Create histogram
fig = px.histogram(gcPBM_myc_final, x='Log Intensity', color='Probe', width=1100, height=600)

# Adjust opacity of bars
opacities = {'Unbound': 0.5, 'Bound': 1}
for trace in fig.data:
    category = trace.name
    trace.marker.opacity = opacities.get(category, 1)  # Default to 1 if category is not in opacities

# Add vertical dashed threshold lines
fig.add_shape(
    go.layout.Shape(type="line", x0=7.7, x1=7.7, y0=0, y1=1, 
                    line=dict(color="black", width=2, dash="dash"), 
                    xref="x", yref="paper")
)
fig.add_shape(
    go.layout.Shape(type="line", x0=9, x1=9, y0=0, y1=1, 
                    line=dict(color="black", width=2, dash="dash"), 
                    xref="x", yref="paper")
)

# Add background color shading
fig.add_shape(
    go.layout.Shape(type="rect", x0=min(gcPBM_myc_final['Log Intensity'])-0.1, x1=7.7, 
                    y0=0, y1=1, fillcolor="green", opacity=0.2, layer="below", 
                    xref="x", yref="paper")
)
fig.add_shape(
    go.layout.Shape(type="rect", x0=7.7, x1=9, 
                    y0=0, y1=1, fillcolor="orange", opacity=0.2, layer="below", 
                    xref="x", yref="paper")
)
fig.add_shape(
    go.layout.Shape(type="rect", x0=9, x1=max(gcPBM_myc_final['Log Intensity']), 
                    y0=0, y1=1, fillcolor="red", opacity=0.2, layer="below", 
                    xref="x", yref="paper")
)

# Add dummy traces for legend (invisible markers for color legend)
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode="markers",
    marker=dict(size=25, color="green", opacity=0.5),
    name="No Binding"
))
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode="markers",
    marker=dict(size=25, color="orange", opacity=0.5),
    name="Weak Binding"
))
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode="markers",
    marker=dict(size=25, color="red", opacity=0.5),
    name="Strong Binding"
))

# Update layout with LaTeX font, larger text, and smaller legend inside the plot
fig.update_layout(
    title=r"Distribution of Log-Transformed Binding Intensities",
    xaxis_title=r"Log Fluorescence Intensity",
    yaxis_title=r"Count",
    font=dict(family="Latex", size=30),
    showlegend=True,
    legend=dict(
        x=0.65,  # Position inside the figure
        y=0.95,
        title='',
        font=dict(size=30),  # Smaller legend font
        bgcolor="rgba(255, 255, 255, 0.7)"  # Light background for readability
    )
)

fig.show()


### Comparative Interpretation: Myc–Max vs Max–Max Binding Intensity Distributions

This plot compares the log-transformed fluorescence intensity distributions of DNA probes bound by two different protein dimers: Myc–Max and Max–Max. Both plots were generated from gcPBM data, but they reveal significantly different binding behaviors.


#### Max–Max: Broad, High-Affinity Binding

- The Max–Max distribution is sharply peaked in the strong binding region (Log Intensity ≥ 9.0), with a large number of probes clustered here.
- This indicates that Max–Max binds to a broad and diverse range of DNA sequences with high affinity.
- Very few probes fall in the low-intensity (unbound) region, meaning Max–Max rarely fails to bind.
- Biologically, this makes sense: Max–Max functions as a general DNA-binding scaffold and can homodimerize to bind canonical E-boxes (e.g., CACGTG) as well as other related motifs.


#### Myc–Max: Selective and Discriminative Binding

- The Myc–Max distribution is bell-shaped, peaking around Log Intensity ≈ 8.3–8.5.
- It shows a wider spread of binding intensities and a significant number of unbound probes (Log Intensity ≤ 7.7).
- This suggests that Myc–Max binds selectively — it prefers high-affinity canonical motifs and avoids low-affinity or flanking sequences.
- Functionally, this reflects its role as a transcriptional activator that must bind with specificity to regulate gene expression.


#### Why This Difference Matters

- With the same labeling thresholds (7.7 for unbound, 9.0 for strong), Myc–Max yields a balanced distribution of unbound, weak, and strong probes.
- In contrast, Max–Max results in a skewed dataset dominated by strong binders.
- This has implications for ML training:
  - You may need to adjust thresholds when labeling Max–Max data.
  - Class balancing strategies may be required to avoid model bias.




| Aspect            | Myc–Max                       | Max–Max                        |
|------------------|-------------------------------|-------------------------------|
| Distribution     | Bell-shaped                   | Right-skewed peak ≥ 9.0       |
| Binding pattern  | Selective                     | Broad, high-affinity          |
| Unbound probes   | Many (green zone visible)     | Very few                      |
| ML labels        | Balanced (0/1/2)              | Skewed (mostly class 2)       |
| Biological role  | Transcriptional activator     | General DNA-binding scaffold  |

These distributions reflect the biological roles of the Myc and Max proteins and must be accounted for when designing computational models to classify or predict binding behavior.


## 3. **Stratify Probes**  

In [34]:
# Make seeds consistent for reproducibility
seeds = range(10000)

# Filter probes into three binding‐strength categories
unbound_probes = gcPBM_myc_final[gcPBM_myc_final['Log Intensity'] <= 8]
weak_probes    = gcPBM_myc_final[(gcPBM_myc_final['Log Intensity'] >  8) & (gcPBM_myc_final['Log Intensity'] < 9)]
strong_probes  = gcPBM_myc_final[gcPBM_myc_final['Log Intensity'] >= 9]

def sample_probes(df, num_samples):
    """
    Evenly sample `num_samples` probes from `df` across 0.1-wide log‐intensity bins,
    ensuring no repeated motifs.  
    Uses a rotating list of k-mer columns (6mer, 8mer, 10mer, 12mer) to break ties
    when a bin runs out of unique motifs.
    Random draws are seeded by `seeds[i]` for reproducibility.
    """
    motif_col    = iter(['6mer', '8mer', '10mer', '12mer'])
    samples      = []  
    
    # define 0.1-wide bins
    bins = np.arange(
        round(df['Log Intensity'].min(), 1),
        round(df['Log Intensity'].max(), 1) + 0.1,
        0.1
    )
    df = df.copy()
    df['bin'] = pd.cut(df['Log Intensity'], bins)
    
    i     = 0
    motif = next(motif_col)
    while i < num_samples:
        len_motifs = []
        for br in df['bin'].cat.categories:
            group = df[df['bin'] == br]
            uniques = group.loc[~group[motif].isin(
                pd.concat(samples)[motif] if samples else []
            ), motif].unique()
            len_motifs.append(len(uniques))
            if len(uniques):
                pick = group[group[motif].isin(uniques)].sample(
                    n=1, random_state=seeds[i]
                )
                samples.append(pick)
                i += 1
                if i >= num_samples:
                    break
        if all(l == 0 for l in len_motifs):
            try:
                motif = next(motif_col)
            except StopIteration:
                break

    if samples:
        result = pd.concat(samples, ignore_index=True)
        result.drop(columns='bin', inplace=True)
        return result
    else:
        # no samples found
        return pd.DataFrame(columns=df.columns.drop('bin'))

# Sample evenly across each binding category
unbound_samples = sample_probes(unbound_probes, 33)
weak_samples    = sample_probes(weak_probes,    33)
strong_samples  = sample_probes(strong_probes,  33)

# Combine and save final balanced set of 99 probes
final_sample = pd.concat([unbound_samples, weak_samples, strong_samples], ignore_index=True)
final_sample.to_csv('dataset_old.csv', index=False)

final_sample

,Name,SEQUENCE,Mean Intensity,Probe,Log Intensity,12mer,10mer,8mer,6mer
0,Unbound_Myc2_119,CATTAAATGGCACCACAATCTTAAATGCCCATCCAG,2915,Unbound,7.977625,CCACAATCTTAA,CACAATCTTA,ACAATCTT,CAATCT
1,Unbound_Myc1_105,GAGAATGGGGTGAACCCGGGAGGCGGAGCTTGCAGT,2933,Unbound,7.983781,AACCCGGGAGGC,ACCCGGGAGG,CCCGGGAG,CCGGGA
2,Unbound_Myc1_115,TTGGCAGCGGACGTCCACCCGGCCACCTCGCCCTGC,2852,Unbound,7.955776,GTCCACCCGGCC,TCCACCCGGC,CCACCCGG,CACCCG
3,Unbound_Myc1_073,TGAGTACAGCTGCGCCCCGCTGTGCGCCCGGTGGGA,2937,Unbound,7.985144,CGCCCCGCTGTG,GCCCCGCTGT,CCCCGCTG,CCCGCT
4,Unbound_Myc1_134,GACTGGACCCCAGAATTGTCCCGGCTTTCTCCCGAG,2974,Unbound,7.997663,GAATTGTCCCGG,AATTGTCCCG,ATTGTCCC,TTGTCC
...,...,...,...,...,...,...,...,...,...
67,Bound_021269_000_100,TGGCTGGCCTGTCTACACGTGTTCTTCCAGGGAAAT,23788,Bound,10.076937,CTACACGTGTTC,TACACGTGTT,ACACGTGT,CACGTG
68,Bound_002527_111_000,TTGGAGTGGCCTCTCCACGTGGCCTAGGCTTCCTGG,25711,Bound,10.154674,CTCCACGTGGCC,TCCACGTGGC,CCACGTGG,CACGTG
69,Bound_009861_001_001,CCCTCCGTGGCTGAACACGTGGGGACAGGGAGTACA,27063,Bound,10.205923,GAACACGTGGGG,AACACGTGGG,ACACGTGG,CACGTG
70,Bound_002998_110_000,GTGGAAAGAGGCACCCACGTGTTCTGATCCATCTGT,30040,Bound,10.310285,ACCCACGTGTTC,CCCACGTGTT,CCACGTGT,CACGTG


In [35]:
old_df = pd.read_csv('dataset_old.csv')
def relabel(row):
    if row['Log Intensity'] <= 7.7:
        return 'unbound'
    elif row['Log Intensity'] < 9:
        return 'weak'
    else:
        return 'strong'
old_df['new_label'] = old_df.apply(relabel, axis=1)

counts = old_df['new_label'].value_counts()
print("Before new dataset: ",counts)

# Filter out any sequences that are already in old_df
all_old_seqs = set(old_df['SEQUENCE'])
unbound_probes_filtered = unbound_probes[~unbound_probes['SEQUENCE'].isin(all_old_seqs)]
weak_probes_filtered = weak_probes[~weak_probes['SEQUENCE'].isin(all_old_seqs)]
strong_probes_filtered = strong_probes[~strong_probes['SEQUENCE'].isin(all_old_seqs)]

#sample from the filtered set
target_unbound = 78
target_weak = 45
target_strong = 45

new_unbound_samples = sample_probes(unbound_probes_filtered, target_unbound - counts['unbound'])
new_unbound_samples['new_label'] = 'unbound'
new_weak_samples = sample_probes(weak_probes_filtered, target_weak - counts['weak'])
new_weak_samples['new_label'] = 'weak'
new_strong_samples = sample_probes(strong_probes_filtered, target_strong - counts['strong'])
new_strong_samples['new_label'] = 'strong'

final_df = pd.concat([old_df, new_unbound_samples, new_weak_samples, new_strong_samples], ignore_index=True)
print("After new dataset: ", final_df['new_label'].value_counts())

final_df.to_csv('dataset.csv', index=False)

# Combine all new sequences (i.e., those sampled from the filtered sets) into one DataFrame
new_sequences = pd.concat([new_unbound_samples, new_weak_samples, new_strong_samples], ignore_index=True)


Before new dataset:  weak      39
strong    33
Name: new_label, dtype: int64


KeyError: 'unbound'

## 4. Save the Final Dataset

In [15]:
threshold = 7.7 # Threshold for binding/not binding

exp = final_df[['SEQUENCE','Log Intensity']].rename(
    columns={'SEQUENCE':'sequence','Log Intensity':'bind_avg'}
)

# multiclass labels: 0=weak,1=medium,2=strong
exp['binding_type'] = exp['bind_avg'].apply(
    lambda x: 0 if x<=threshold else 1 if x<9 else 2
)
# binary: 0=binder (>=threshold), 1=non-binder
exp['improving'] = exp['bind_avg'].apply(lambda x: 0 if x>=threshold else 1)
# regression: center around threshold
exp['bind_avg'] = exp['bind_avg'] - threshold


In [18]:
# Save files
exp.to_csv('exp_data_all.csv', index=False)

# quick plots
px.histogram(exp, x='bind_avg', nbins=41, title='bind_avg').show()
px.scatter(exp.sort_values('bind_avg').reset_index(), y='bind_avg',
           title='bind_avg').show()